# AirBNB Price Analyis

1. Webscrape AirBNB page to find absolute location of stay
2. Convert absolute location to address
3. Search for address on Zillow
4. Webscrape Zillow of stay to find Zestimate or actual price
5. Compare AirBNB price with Zillow price, analyze the stay premium

### Imports

In [85]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
from googlesearch import search

### Step 1: Webscraping AirBNB Rental

In [110]:
listing_url = "https://www.airbnb.com/rooms/633490307738150521"
options = Options()
#options.add_argument("-headless")
driver = webdriver.Firefox(options=options)
driver.get(listing_url)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [105]:
def getNumbers(w):
    nums = ""
    for char in w:
        if (char.isnumeric()):
            nums += char
    return int(nums)

elem = driver.find_element("id", "data-deferred-state").get_attribute("innerHTML")
price, info
while (not price or not info):
    try:
        price = driver.find_elements("class name", "_tyxjp1")[1].text.strip()
        info = [getNumbers(_) for _ in driver.find_element("class name","lgx66tx").text.split("·")]
    except:
        pass
numGuests = info[0]
numBr = info[1]
numBed = info[2]
numBath =  info[3]
lastChars,lng,lat = "","",""
i = 0
for char in elem:
    if (len(lastChars) == 6):
        lastChars = lastChars[1:6] + char
    else:
        lastChars = lastChars + char
    if(lastChars == "\"lat\":"):
        lat = elem[i+1:i+12]
    elif(lastChars == "\"lng\":"):
        lng = elem[i+1:i+12]
    i+=1
while(not lat[len(lat)-1].isnumeric() or not lng[len(lng)-1].isnumeric()):
    if(lat[len(lat)-1].isnumeric()):
        lng = lng[:len(lng)-1]
    else:
        lat = lat[:len(lat)-1]

print(price, "per night")
print(numBr,("Bedroom," if numBr == 1 else "Bedrooms,"),numBath,("Bathroom" if numBath == 1 else "Bathrooms"))
print(lat,lng,sep = ", ")

$109 per night
1 Bedroom, 1 Bathroom
38.95711, -77.41784


### Step 2. Getting AirBNB Address

In [ ]:
driver.get("https://www.gps-coordinates.net/gps-coordinates-converter")
latElem = driver.find_element("id","latitude")
lngElem = driver.find_element("id","longitude")
latElem.clear()
latElem.send_keys(lat)
lngElem.clear()
lngElem.send_keys(lng)
addresses = driver.find_element("id","address").get_attribute("value")
address = addresses
i = 0
for char in addresses:
    if(char == "–" or char == "-"):
        address = addresses[i+1:]
        break
    i+=1
print(address)

### Step 3. Getting Associated Zillow Webpage

In [84]:
results = []
for result in search(address + "zillow", num_results = 3):
    results.append(result)
driver.get(results[0])
